<h3 align = "center">Welcome to the text-based Black Jack game!</h3><hr>


* 1 to 6 players
* Players may participate in multiple rounds (hands)
* Each player starts with 100 "chips" to use for betting. These winnings/loses are carried to the next round.
* The dealer's turns will automatically be taken and displayed
* Players will have the option to split their hand if they have two cards of the same value

<h3>0. Import Libraries</h3>

In [ ]:
import random

<h3>1. Create Classes:</h3>Card, Deck, Hand, and Player 

In [ ]:
# Prepare cards
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

class Card:   
    
    def __init__(self, suit, rank):        
        self.suit = suit
        self.rank = rank       
        self.value = values[rank] # value assigns an integer to each rank, so cards can be compared
        
    def __str__(self):       
        return self.rank + ' of ' + self.suit

    
class Deck:    
    def __init__(self):
        
        # Begin with empty deck of cards
        self.deck = []
        
        # Create all 52 cards
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))

        # Automatically shuffle deck
        random.shuffle(self.deck)            
    
    def __str__(self):
        
        deck_comp = ''
        for card in self.deck:
            deck_comp = deck_comp + f'{card}\n'
        return 'The deck has:\n' + deck_comp
        
    def deal(self):
        return self.deck.pop()
    
    
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list
        self.value = 0   # start with zero value
        self.aces = 0    # Keep track of how many aces
    
    def add_card(self,card):
        
        self.cards.append(card)
        self.value += card.value
        
        if card.rank == 'Ace':
            self.aces += 1
    
    def adjust_for_ace(self):
        # Remove 10 from value of Ace if a hand busts. Remove 1 from Ace count to recognize
        # How many Aces remain unchanged
        if self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1
            print("You went over 21, so your Ace is now worth 1.")
            
    def clear(self):
        self.cards = []
        self.value = 0   
        self.aces = 0 
            
    def __str__(self):
        
        hand_comp = ''  
        
        for card in self.cards:
            hand_comp = hand_comp + f"{card}\n" 
            
        return hand_comp
    

class Player():    
    def __init__(self, name, chips = 100, current_bet = 0):
        
        self.name = name
        self.chips = chips
        self.bet = current_bet
        self.hand = Hand()
                
    def __str__(self):
        return f"{self.name}"

<h3>2. Create all game functions:</h3> player select, deal to all players, place bet, hit or stand, split, player turn,<br> dealer turn, show some (dealer), show all (player), show hand (player split), check winners

In [ ]:
def player_select():
    
    while True:

        # Select number of players
        try:
            number_of_players = int(input("How many players will be joining? "))
            
            if 0 < number_of_players < 7 :           
                break
            else:
                print("Please select 1 to 6 players.\n")
                pass               
            
        except:
            print("Please enter an integer value.\n")
            pass
        
    # Select a name for each player and create the player instances.
    global player_list
    player_list = []
    
    for i in range(1,number_of_players+1):
        name = input(f"Player {i}, what is your name? ")
        player = Player(f"{name}")
        player_list.append(player)

        
# Deal two cards to each player and the dealer in proper order (one card at a time)
def deal_to_all(deck, dealer, player_list):
    
    for i in range(0,2):
        dealer.hand.add_card(deck.deal())
        
        for player in player_list:
            player.hand.add_card(deck.deal())

            
def place_bet(player):

    print(f"\n{player}, you currently have " + str(player.chips) + " chips")
    
    # Loop until acceptable bet is made
    while True:
        
        try:
            # Bet must be an integer
            bet = int(input("How much would you like to bet? "))

        except:
            print("Please enter a valid amount.")
            
        else:

            # Player must have enough chips. if bet is acceptable, return bet
            if bet <= player.chips and bet > 0:
                player.bet = bet
                break
                
            elif bet <= 0:
                print('Please enter an integer greater than zero.')
                
            else:
                print("You don't have enough chips for that bet!")     
                
                
# Passing hand as argument because the player may choose to split
def hit_or_stand(deck, player, hand):

    show_hand(player, hand)
    
    while hit_or_stand:

        player_choice = input("Would you like to 'Hit' or 'Stand'? ")

        if player_choice.lower() == 'hit':

            # Add_card already keeps track of the total hand value
            hand.add_card(deck.deal())
            show_hand(player, hand)
            
            # Check for 21 or bust           
            hand.adjust_for_ace()
            
            if hand.value > 21:
                print("That's a bust!")
                return 0 

            elif hand.value == 21:
                print("That's 21!")
                break
        
        elif player_choice.lower() == 'stand':
            break

        else:
            print("Please enter a valid selection.")
            pass
              
    return hand.value


# Offer the opporunity for a player to split their hand, if applicable
def split(player):

    global current_hands
    current_hands = []
    
    if player.hand.cards[0].rank == player.hand.cards[1].rank:
        split = input("Would you like to split your hand (Yes or No)? ")
        
    # If player chooses to split, then split the original hand to a nested list    
        if split.lower() == 'yes':
            
            hand_1 = Hand()
            hand_1.add_card(player.hand.cards[0])
            hand_2 = Hand()
            hand_2.add_card(player.hand.cards[1])

            current_hands.append(hand_1)
            current_hands.append(hand_2)
                  
        else:           
            current_hands.append(player.hand)
            
    else:
        current_hands.append(player.hand)


# Contains a player's turn in its entirety and will finish with player.value = best value. 
def player_turn(deck,player, current_hands):
            
    # Player hits or stands until finished    
    best_value = 0
    
    for hand in current_hands:
        current_value = hit_or_stand(deck, player, hand)
            
        # Player takes best value of each hand (applicable for splits)
        if current_value > best_value:
            best_value = current_value
            pass
        
        elif current_value <= best_value:
            pass
        
    player.hand.value = best_value
    

# Automated dealer's turn
def dealer_turn(deck, dealer):
    
        # show all dealer cards for the first time
        show_all(dealer)
    
        while True:
            
            if 17 <= dealer.hand.value <= 21:
                break
                
            elif dealer.hand.value > 21:
                print("The dealer busted!")
                dealer.hand.value = 0
                break
                
            else:
                dealer.hand.add_card(deck.deal())
                show_all(dealer)
                pass

            
# Show only one of the dealer's cards
def show_some(dealer):
    print(f"\nThe dealer's first card is {dealer.hand.cards[0]} and the second card is still hidden.")

    
# Show player's cards
def show_all(player):
    print(f"\n{player} has ")
    for card in player.hand.cards:
        print(f"{card}")

        
# Show player's cards when they have split to multiple 'hands'   
def show_hand(player,hand):
    print(f"\n{player}'s current hand is:")
    for card in hand.cards:
          print(f'{card}')

            
def check_winners():
    
    for player in player_list:
        
        if player.hand.value > dealer.hand.value:
            player.chips += player.bet
            print(f"\n{player}, you won! You now have {player.chips} chips.\n")

        elif player.hand.value == dealer.hand.value:
            print(f"\n{player}, you tied! You still have {player.chips} chips.\n")

        elif player.hand.value < dealer.hand.value:
            player.chips -= player.bet
            print(f"\n{player}, you lost! You now have {player.chips} chips.\n")
            
            #remove players who no longer have any chips
            if player.chips == 0:
                print(f"\n{player} is out of chips! You are no longer in the game\n")
                player_list.remove(player)
            else:
                pass

<h3>3. Play on!</h3>
<h4>(Game Logic)</h4>

In [ ]:
print("Welcome to Black Jack!")
    
# Player set up: Number of players, player names, and start with 100 chips
player_select()
dealer = Player('Dealer')
        
# May the game begin!
playing = True
    
while playing:
        
    # Create/shuffle the deck, deal two cards to each player (in order)
    deck = Deck()
    deal_to_all(deck, dealer, player_list)    

    # Prompt each player for their bet
    for player in player_list:
        player.current_bet = place_bet(player)
               
    # Show cards (but keep one dealer card hidden)
    show_some(dealer)

    for player in player_list:
        show_all(player)

    # All players take their turn
    for player in player_list:
        
        print(f"\n{player}, it's your turn!")
        
        #check for a split
        split(player)
               
        #Player can hit, stand, or double down
        player_turn(deck, player, current_hands)
        
    # Dealer's turn
    dealer_turn(deck, dealer)

    #check for win/lose/tie for each player
    check_winners()
               
    # Check for remaining players
    if player_list != []:
        pass
    else:
        print('Everyone is out of chips! The game is over.')
        break
      
    # Ask to play again    
    while True:
        
        play_again = input("Play again? (Yes or No) ")

        if play_again.lower() == 'yes':
            break
        elif play_again.lower() == 'no':
            print('\nBye!')
            playing = False
            break
        else:
            print("Please enter a valid response.")
            pass
     
    # If playing == true, clear all hands
    for player in player_list:
        player.hand.clear()
    
    dealer.hand.clear()
    
    pass